In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from typing import List

In [ ]:
CASAS_DE_APOSTA_BR = [
    'Dafabet',
    'Betfair',
    'Betway',
    'Betano',
    'Bet365',
    'Galera Bet',
    'Betsson',
    '22BET',
    'Betwinner',
    'Rabona',
    '888sport',
    'LeoVegas',
    'Rivalo',
    'Parimatch',
    'Sportaza',
    'Betkwiff',
    'Amuletobet',
    '20BET',
    'Powbet',
    'FezBet',
    'Campeonbet',
    'Melbet',
    'Pixbet',
    'BetWarrior',
    'Bet7',
    'Megapari',
    'KTO',
    'Vbet',
    'CampoBet',
    'CyberBet',
    'LVBET',
    'Betmaster',
    'Bodog',
    'Fansbet',
    'Betmotion'
]

SUBURL = '/soccer/brazil/serie-a/'
CASAS_DE_APOSTA_BR = list(map(lambda s: s.lower(), CASAS_DE_APOSTA_BR))

available_games = list()
sites = dict()

In [ ]:
# Baixa as casas de aposta brasileiras para um determinado jogo
def retrieve_bet_houses_info_from_spec_game(parsed_text: List[str]):
    global sites
    inc = 0
    try:
        while True:
            casa_aposta, odd_a, empate, odd_b, payout = parsed_text[5+inc:10+inc]
            casa_aposta = casa_aposta.lower()
            if 'Log in to display the odds!' in [casa_aposta, odd_a, empate, odd_b, payout]:
                break

            if casa_aposta in CASAS_DE_APOSTA_BR:
                sites[casa_aposta] = {
                    "odd_a":    odd_a,
                    "Empate":   empate,
                    "odd_b":    odd_b,
                    "payout":   payout
                }
            inc += 5
    except:
        pass
    
    # print("Sites obtidos: ", sites)
    return pd.DataFrame(sites).T

In [ ]:
def retrieve_game_parsed_text(URL: str):
    webElement = None
    driver = webdriver.Chrome()
    try:
        driver.maximize_window()
        driver.get(URL)
        time.sleep(10)
        webElement = driver.execute_script('return document.getElementsByClassName("table-main detail-odds sortable")[0].outerHTML')
        title = driver.execute_script('return document.title')
    except Exception as e:
        print(f"Um erro ocorreu ao executar o arquivo: \nLog: {e}")
    
    assert isinstance(webElement, str) and isinstance(webElement, str) 
    print(title)

    SoupElement = BeautifulSoup(webElement, 'html.parser')
    raw_text = [text for text in SoupElement.stripped_strings]
    # print(raw_text)

    driver.close()
    return retrieve_bet_houses_info_from_spec_game(raw_text)

In [ ]:
url = "https://www.oddsportal.com/soccer/brazil/serie-a"
driver = webdriver.Chrome()
webElement = None

try:
    driver.maximize_window()
    driver.get(url)
    time.sleep(10)
    webElement = driver.execute_script('return document.getElementsByClassName("table-main")[0].outerHTML')
    print("Jogos baixados com sucesso.")
except Exception as e:
    print(f"Um erro ocorreu ao executar o arquivo: \nLog: {e}")
    driver.close()
    exit(-1)
finally:
    driver.close()


assert isinstance(webElement, str)

SoupElement = BeautifulSoup(webElement, 'html.parser')
raw_text = [text for text in SoupElement.stripped_strings]

# Obtém todos os jogos disponíveis na tabela
for href in SoupElement.find_all('a', href=True):
    if href['href'][:23]  == '/soccer/brazil/serie-a/' and href['href'] != SUBURL:
        available_games.append(href['href'])

# print(available_games)
for i, link in enumerate(available_games):
    print(retrieve_game_parsed_text("https:www.oddsportal.com" + link))1